In [ ]:
# !pip3 install numpy==1.23.5

In [ ]:
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore')
import configparser
import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np

import itertools
import os
import time
import json
import torch
import torch.nn.functional as F
# from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DistributedSampler, DataLoader
import torch.multiprocessing as mp
from torch.distributed import init_process_group
from torch.nn.parallel import DistributedDataParallel

import sys
sys.path.append("../")
from env import AttrDict, build_env
from meldataset import MelDataset, mel_spectrogram, get_dataset_filelist
from models import Generator, MultiPeriodDiscriminator, MultiScaleDiscriminator, feature_loss, generator_loss,\
    discriminator_loss
from utils import plot_spectrogram, scan_checkpoint, load_checkpoint, save_checkpoint

torch.backends.cudnn.benchmark = True

sys.path.append("../../cuhksz-phd/sho_util/pyfiles/")
from pytorch import cuda2numpy
from basic import plot_spectrogram
from sound import play_audio

a = configparser.ConfigParser()

In [ ]:
base_dir = "../"
# data_dir = f"{base_dir}LibriTTS/"
data_dir = f"{base_dir}LibriTTS_ARCTIC_SPAccenT/"
a.group_name = None
a.input_wavs_dir = "/mntcephfs/" 
a.input_mels_dir = "/mntcephfs/"
a.input_training_file = f'{data_dir}/training.txt'
a.input_validation_file = f'{data_dir}/validation.txt'

# a.checkpoint_path = f'/mntcephfs/data/audiow/shoinoue/Model/models/hifigan/LibriTTS' # only libritts
# a.checkpoint_path = f'/mntcephfs/data/audiow/shoinoue/Model/models/hifigan/LibriTTS2' # alldatasets
a.checkpoint_path = f'/mntcephfs/data/audiow/shoinoue/Model/models/hifigan/LibriTTS3' # onlylibritts + longer segmentsize

# a.config = f'{base_dir}config_16kHz.json'
a.config = f'{base_dir}config_16kHz2.json' # longer segmentsize

a.training_epochs = 10000
# a.stdout_interval = 5
a.stdout_interval = 100
a.checkpoint_interval = 1000
a.summary_interval = 100
a.validation_interval = 1000
# a.validation_interval = 10
a.fine_tuning = False

scaler = None
import joblib
scaler_filename = f"../../seq2seq-vc/notebooks/ckpts/scalers/LibriTTS-R_hifiganmel.save"
scaler = joblib.load(scaler_filename)

- Preparation

In [ ]:
with open(a.config) as f:
    data = f.read()

json_config = json.loads(data)
h = AttrDict(json_config)
build_env(a.config, 'config.json', a.checkpoint_path)
h["batch_size"] = 16

torch.manual_seed(h.seed)
torch.cuda.manual_seed(h.seed)
h.num_gpus = torch.cuda.device_count()
h.batch_size = int(h.batch_size / h.num_gpus)
print('Batch size per GPU :', h.batch_size)

rank = 0

In [ ]:
if h.num_gpus > 1:
    init_process_group(backend=h.dist_config['dist_backend'], init_method=h.dist_config['dist_url'],
                       world_size=h.dist_config['world_size'] * h.num_gpus, rank=rank)

torch.cuda.manual_seed(h.seed)
device = torch.device('cuda:{:d}'.format(rank))
generator = Generator(h).to(device)
mpd = MultiPeriodDiscriminator().to(device)
msd = MultiScaleDiscriminator().to(device)

In [ ]:
if rank == 0:
    # print(generator)
    os.makedirs(a.checkpoint_path, exist_ok=True)
    print("checkpoints directory : ", a.checkpoint_path)

if os.path.isdir(a.checkpoint_path):
    cp_g = scan_checkpoint(a.checkpoint_path, 'g_')
    cp_do = scan_checkpoint(a.checkpoint_path, 'do_')

steps = 0
if cp_g is None or cp_do is None:
    state_dict_do = None
    last_epoch = -1
else:
    state_dict_g = load_checkpoint(cp_g, device)
    state_dict_do = load_checkpoint(cp_do, device)
    generator.load_state_dict(state_dict_g['generator'])
    mpd.load_state_dict(state_dict_do['mpd'])
    msd.load_state_dict(state_dict_do['msd'])
    steps = state_dict_do['steps'] + 1
    last_epoch = state_dict_do['epoch']

if h.num_gpus > 1:
    generator = DistributedDataParallel(generator, device_ids=[rank]).to(device)
    mpd = DistributedDataParallel(mpd, device_ids=[rank]).to(device)
    msd = DistributedDataParallel(msd, device_ids=[rank]).to(device)

optim_g = torch.optim.AdamW(generator.parameters(), h.learning_rate, betas=[h.adam_b1, h.adam_b2])
optim_d = torch.optim.AdamW(itertools.chain(msd.parameters(), mpd.parameters()),
                            h.learning_rate, betas=[h.adam_b1, h.adam_b2])

if state_dict_do is not None:
    optim_g.load_state_dict(state_dict_do['optim_g'])
    optim_d.load_state_dict(state_dict_do['optim_d'])

scheduler_g = torch.optim.lr_scheduler.ExponentialLR(optim_g, gamma=h.lr_decay, last_epoch=last_epoch)
scheduler_d = torch.optim.lr_scheduler.ExponentialLR(optim_d, gamma=h.lr_decay, last_epoch=last_epoch)

training_filelist, validation_filelist = get_dataset_filelist(a)

trainset = MelDataset(training_filelist, h.segment_size, h.n_fft, h.num_mels,
                      h.hop_size, h.win_size, h.sampling_rate, h.fmin, h.fmax, n_cache_reuse=0,
                      shuffle=False if h.num_gpus > 1 else True, fmax_loss=h.fmax_for_loss, device=device,
                      fine_tuning=a.fine_tuning, base_mels_path=a.input_mels_dir, base_wavs_path=a.input_wavs_dir,
                      scaler=scaler,
                     )

train_sampler = DistributedSampler(trainset) if h.num_gpus > 1 else None
# trainset.fine_tuning = True
train_loader = DataLoader(trainset, num_workers=h.num_workers, shuffle=False,
                          sampler=train_sampler,
                          batch_size=h.batch_size,
                          pin_memory=True,
                          drop_last=True)

if rank == 0:
    validset = MelDataset(validation_filelist, h.segment_size, h.n_fft, h.num_mels,
                          h.hop_size, h.win_size, h.sampling_rate, h.fmin, h.fmax, False, False, n_cache_reuse=0,
                          fmax_loss=h.fmax_for_loss, device=device, fine_tuning=a.fine_tuning,
                          base_mels_path=a.input_mels_dir, base_wavs_path=a.input_wavs_dir, 
                          scaler=scaler,
                         )
    # validset.fine_tuning = True
    validation_loader = DataLoader(validset, num_workers=1, shuffle=False,
                                   sampler=None,
                                   batch_size=1,
                                   pin_memory=True,
                                   drop_last=True)

    # sw = SummaryWriter(os.path.join(a.checkpoint_path, 'logs'))
    
# start = 80
valsample = validset[0][0]#[:,start:start+32]
trainsample = trainset[1][0]

In [ ]:
def transform(x):
    mean_ = torch.tensor(scaler.mean_.reshape(1, -1, 1)).to(x.device)
    scale_ = torch.tensor(scaler.scale_.reshape(1, -1, 1)).to(x.device)
    return (x-mean_)/scale_

def inverse_transform(x):
    mean_ = torch.tensor(scaler.mean_.reshape(1, -1, 1)).to(x.device)
    scale_ = torch.tensor(scaler.scale_.reshape(1, -1, 1)).to(x.device)
    return (x*scale_+mean_)

In [ ]:
%matplotlib inline
play = True
plot = True
generator.train()
mpd.train()
msd.train()
loss_train = []
err_train = []
err_val = []
for epoch in range(max(0, last_epoch), a.training_epochs):
    if rank == 0:
        start = time.time()
        print("Epoch: {}".format(epoch+1))

    if h.num_gpus > 1:
        train_sampler.set_epoch(epoch)

    for i, batch in enumerate(train_loader):
        if rank == 0:
            start_b = time.time()
        x, y, _, y_mel = batch
        x = torch.autograd.Variable(x.to(device, non_blocking=True))
        y = torch.autograd.Variable(y.to(device, non_blocking=True))
        y_mel = torch.autograd.Variable(y_mel.to(device, non_blocking=True))
        y = y.unsqueeze(1)

        y_g_hat = generator(x)
        y_g_hat_mel = mel_spectrogram(y_g_hat.squeeze(1), h.n_fft, h.num_mels, h.sampling_rate, h.hop_size, h.win_size,
                                      h.fmin, h.fmax_for_loss)
        if scaler!=None:
            y_g_hat_mel = transform(y_g_hat_mel)

        optim_d.zero_grad()

        # MPD
        y_df_hat_r, y_df_hat_g, _, _ = mpd(y, y_g_hat.detach())
        loss_disc_f, losses_disc_f_r, losses_disc_f_g = discriminator_loss(y_df_hat_r, y_df_hat_g)

        # MSD
        y_ds_hat_r, y_ds_hat_g, _, _ = msd(y, y_g_hat.detach())
        loss_disc_s, losses_disc_s_r, losses_disc_s_g = discriminator_loss(y_ds_hat_r, y_ds_hat_g)

        loss_disc_all = loss_disc_s + loss_disc_f

        loss_disc_all.backward()
        optim_d.step()

        # Generator
        optim_g.zero_grad()

        # L1 Mel-Spectrogram Loss
        loss_mel = F.l1_loss(y_mel, y_g_hat_mel) * 45

        y_df_hat_r, y_df_hat_g, fmap_f_r, fmap_f_g = mpd(y, y_g_hat)
        y_ds_hat_r, y_ds_hat_g, fmap_s_r, fmap_s_g = msd(y, y_g_hat)
        loss_fm_f = feature_loss(fmap_f_r, fmap_f_g)
        loss_fm_s = feature_loss(fmap_s_r, fmap_s_g)
        loss_gen_f, losses_gen_f = generator_loss(y_df_hat_g)
        loss_gen_s, losses_gen_s = generator_loss(y_ds_hat_g)
        loss_gen_all = loss_gen_s + loss_gen_f + loss_fm_s + loss_fm_f + loss_mel

        loss_gen_all.backward()
        optim_g.step()

        if rank == 0:
            # STDOUT logging
            if steps % a.stdout_interval == 0:
                with torch.no_grad():
                    mel_error = F.l1_loss(y_mel, y_g_hat_mel).item()

            # checkpointing
            if steps % a.checkpoint_interval == 0 and steps != 0:
                checkpoint_path = "{}/g_{:08d}".format(a.checkpoint_path, steps)
                save_checkpoint(checkpoint_path,
                                {'generator': (generator.module if h.num_gpus > 1 else generator).state_dict()})
                checkpoint_path = "{}/do_{:08d}".format(a.checkpoint_path, steps)
                save_checkpoint(checkpoint_path, 
                                {'mpd': (mpd.module if h.num_gpus > 1
                                                     else mpd).state_dict(),
                                 'msd': (msd.module if h.num_gpus > 1
                                                     else msd).state_dict(),
                                 'optim_g': optim_g.state_dict(), 'optim_d': optim_d.state_dict(), 'steps': steps,
                                 'epoch': epoch})


            # Validation
            if steps % a.validation_interval == 0 and steps != 0:
                generator.eval()
                torch.cuda.empty_cache()
                val_err_tot = 0
                with torch.no_grad():
                    for j, batch in enumerate(validation_loader):
                        x, y, _, y_mel = batch
                        y_g_hat = generator(x.to(device))
                        y_mel = torch.autograd.Variable(y_mel.to(device, non_blocking=True))
                        y_g_hat_mel = mel_spectrogram(y_g_hat.squeeze(1), h.n_fft, h.num_mels, h.sampling_rate,
                                                      h.hop_size, h.win_size,
                                                      h.fmin, h.fmax_for_loss)
                        if scaler!=None:
                            y_g_hat_mel = transform(y_g_hat_mel)
                        val_err_tot += F.l1_loss(y_mel, y_g_hat_mel).item()

                        if j <= 4:
                            y_hat_spec = mel_spectrogram(y_g_hat.squeeze(1), h.n_fft, h.num_mels,
                                                         h.sampling_rate, h.hop_size, h.win_size,
                                                         h.fmin, h.fmax)

                    val_err = val_err_tot / (j+1)

                generator.train()


        ###### PLOT #######
        if steps % a.stdout_interval == 0:
            clear_output(wait=True)
            with torch.no_grad():
                train_audio = generator(trainsample.unsqueeze(0).to(device))
                train_target = mel_spectrogram(train_audio.squeeze(1), h.n_fft, h.num_mels, h.sampling_rate,
                                              h.hop_size, h.win_size,
                                              h.fmin, h.fmax_for_loss)
                # if scaler!=None:
                    # train_target = transform(train_target)
            with torch.no_grad():
                val_audio = generator(valsample.unsqueeze(0).to(device))
                val_target = mel_spectrogram(val_audio.squeeze(1), h.n_fft, h.num_mels, h.sampling_rate,
                                              h.hop_size, h.win_size,
                                              h.fmin, h.fmax_for_loss)
                # if scaler!=None:
                    # val_target = transform(val_target)
            print('Steps : {:d}, Gen Loss Total : {:4.3f}, Mel-Spec. Error : {:4.3f}, s/b : {:4.3f}'.
                  format(steps, loss_gen_all, mel_error, time.time() - start_b))
            if play:
                play_audio(cuda2numpy(val_audio)[0][0], h.sampling_rate)
            loss_train += [cuda2numpy(loss_gen_all)]
            err_train += [mel_error]
            try:
                err_val += [val_err]
            except NameError:
                pass
            
            length = 2
            width = 3
            fig = plt.figure(figsize=(4*width, 5*length))

            ax = fig.add_subplot(length, width, 1)
            ax.plot(np.arange(len(loss_train))*5, loss_train, label="train")
            ax.set_title("Whole Train Loss")
            ax.set_xlabel("steps (iterations)")
            plt.legend()

            ax = fig.add_subplot(length, width, 2)
            ax.plot(np.arange(len(err_train))*5, err_train, label="train")
            try:
                ax.plot(np.arange(len(err_val))*5, err_val, label="val")
            except NameError:
                pass
            plt.legend()
            ax.set_title("Mel-spectrogram Error")
            ax.set_xlabel("steps (iterations)")

            # Training
            M = cuda2numpy(inverse_transform(trainsample.unsqueeze(0)))[0]
            ax = plot_spectrogram(M, fig, (length, width, 4), title="train: source", title_font=12)
            M = cuda2numpy(train_target)[0]
            ax = plot_spectrogram(M, fig, (length, width, 5), title="train: target", title_font=12)

            # Validation
            M = cuda2numpy(inverse_transform(valsample.unsqueeze(0)))[0]
            ax = plot_spectrogram(M, fig, (length, width, 3), title="val: source", title_font=12)
            M = cuda2numpy(val_target)[0]
            ax = plot_spectrogram(M, fig, (length, width, 6), title="val: target", title_font=12)
            
            if steps % a.checkpoint_interval == 0 and steps != 0:
                # plt.savefig("figures/progress_{:08d}".format(steps), dpi='figure', format=None, metadata=None,
                plt.savefig("figures2/progress_{:08d}".format(steps), dpi='figure', format=None, metadata=None,
                # plt.savefig("figures3/progress_{:08d}".format(steps), dpi='figure', format=None, metadata=None,
                        bbox_inches="tight", pad_inches=0.1,
                        facecolor='auto', edgecolor='auto', backend=None)
            

            if plot:
                plt.show()
        ###################
        
        steps += 1
        
    scheduler_g.step()
    scheduler_d.step()
    

    if rank == 0:
        print('Time taken for epoch {} is {} sec\n'.format(epoch + 1, int(time.time() - start)))

In [ ]:
train_loader